In [48]:
import pandas
import numpy
pandas.options.mode.chained_assignment = None

In [49]:
circulatory_mortality = pandas.read_csv("input/Circulatory Mortality.csv")
respiratory_mortality = pandas.read_csv("input/Respiratory Mortality.csv")
all_causes_mortality = pandas.read_csv("input/All Cause Mortality.csv")
all_data = [circulatory_mortality, respiratory_mortality, all_causes_mortality]

In [50]:
def convert_ids_to_int(df):
    df = df[df["State ANSI"].notnull()]
    df["STATEFP"] = pandas.to_numeric(df["State ANSI"], downcast="integer")
    df["COUNTYFP"] = pandas.to_numeric(df["County ANSI"], downcast="integer")
    return df.drop(columns=["State ANSI", "County ANSI"])

In [51]:
circulatory_mortality = convert_ids_to_int(circulatory_mortality)
respiratory_mortality = convert_ids_to_int(respiratory_mortality)
all_causes_mortality = convert_ids_to_int(all_causes_mortality)

rename_map = {
    "Deaths_0-5": "mort_0-5",
    "Deaths_25+": "mort_25+",
    "Deaths_5-25": "mort_5-25",
}

circulatory_mortality = circulatory_mortality.rename(columns=rename_map)
respiratory_mortality = respiratory_mortality.rename(columns=rename_map)
all_causes_mortality = all_causes_mortality.rename(columns=rename_map)

In [52]:
circulatory_mortality

,mort_0-5,mort_25+,mort_5-25,Population_0-5,Population_25+,Population_5-25,STATEFP,COUNTYFP
0,0.0,157.0,0.0,3191.0,37332.0,14893.0,1,1
1,0.0,631.0,0.0,11660.0,147731.0,49172.0,1,3
2,0.0,81.0,0.0,1352.0,18326.0,6287.0,1,5
3,0.0,73.0,0.0,1274.0,16121.0,5248.0,1,7
4,0.0,184.0,0.0,3353.0,39867.0,14484.0,1,9
...,...,...,...,...,...,...,...,...
3137,0.0,83.0,0.0,3107.0,28510.0,12548.0,56,37
3138,0.0,0.0,0.0,1209.0,17488.0,4494.0,56,39
3139,0.0,25.0,0.0,1634.0,13119.0,6020.0,56,41
3140,0.0,0.0,0.0,474.0,5722.0,2039.0,56,43


In [64]:
circ_and_resp = circulatory_mortality.merge(respiratory_mortality, on=["STATEFP", "COUNTYFP", "Population_0-5", "Population_25+", "Population_5-25"], suffixes=["_circ", "_resp"])

In [68]:
all_data = all_causes_mortality.merge(circ_and_resp, on=["STATEFP", "COUNTYFP", "Population_0-5", "Population_25+", "Population_5-25"])

In [71]:
all_data.to_csv("output/mortality.csv", index=None)